In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Broadcast").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/26 18:58:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark

In [3]:
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=Broadcast>

In [4]:
lookup = {
"TX": "Texas",
"NY": "New York",
"OH": "OHIO",
"CA": "California",
"IL": "Illinois",
"CO": "Colorado",
"AZ": "Arizona"
}

type(lookup)

dict

In [5]:
df =  spark.read.csv("DataSets/uspopulation.csv",header=True,sep="|")
df.show(5)

+---------+-----------+----------+-------------+-----------+------------------+
|2019_rank|       City|State_Code|2019_estimate|2010_Census|            Change|
+---------+-----------+----------+-------------+-----------+------------------+
|        1|New York[d]|        NY|      8336817|    8175133|            0.0198|
|        2|Los Angeles|        CA|      3979576|    3792621|            0.0493|
|        3|    Chicago|        IL|      2693976|    2695598|−0.06%            |
|        4| Houston[3]|        TX|      2320268|    2100263|            0.1048|
|        5|    Phoenix|        AZ|      1680992|    1445632|            0.1628|
+---------+-----------+----------+-------------+-----------+------------------+
only showing top 5 rows



Broadcast variables allow the developers to cache a copy of the read-only variable on each machine/node rather than moving the copy of it with tasks.

In [7]:
broad = sc.broadcast(lookup)
type(broad)

pyspark.broadcast.Broadcast

In [8]:
broad.value["TX"]

#destroy --- Deleting the broadcasted varible from the memory of all the executors (removes the data from driver also)
#dump
#load
#load_from_path
#unpersist --- remove the data from cache memory
#value

'Texas'

In [9]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

In [10]:
def broadFunc(colName):
    return broad.value[colName]

udf_broad_func = udf(broadFunc)


In [11]:
df.withColumn("State Full",udf_broad_func('State_Code')).show(5)

+---------+-----------+----------+-------------+-----------+------------------+----------+
|2019_rank|       City|State_Code|2019_estimate|2010_Census|            Change|State Full|
+---------+-----------+----------+-------------+-----------+------------------+----------+
|        1|New York[d]|        NY|      8336817|    8175133|            0.0198|  New York|
|        2|Los Angeles|        CA|      3979576|    3792621|            0.0493|California|
|        3|    Chicago|        IL|      2693976|    2695598|−0.06%            |  Illinois|
|        4| Houston[3]|        TX|      2320268|    2100263|            0.1048|     Texas|
|        5|    Phoenix|        AZ|      1680992|    1445632|            0.1628|   Arizona|
+---------+-----------+----------+-------------+-----------+------------------+----------+
only showing top 5 rows



Asynchronously delete cached copies of this broadcast on the executors. If the broadcast is used after this is called, it will need to be re-sent to each executor.

In [12]:
broad.unpersist()
df.withColumn("State Full",udf_broad_func('State_Code')).show(5)

+---------+-----------+----------+-------------+-----------+------------------+----------+
|2019_rank|       City|State_Code|2019_estimate|2010_Census|            Change|State Full|
+---------+-----------+----------+-------------+-----------+------------------+----------+
|        1|New York[d]|        NY|      8336817|    8175133|            0.0198|  New York|
|        2|Los Angeles|        CA|      3979576|    3792621|            0.0493|California|
|        3|    Chicago|        IL|      2693976|    2695598|−0.06%            |  Illinois|
|        4| Houston[3]|        TX|      2320268|    2100263|            0.1048|     Texas|
|        5|    Phoenix|        AZ|      1680992|    1445632|            0.1628|   Arizona|
+---------+-----------+----------+-------------+-----------+------------------+----------+
only showing top 5 rows



Destroy all data and metadata related to this broadcast variable. Use this with caution; once a broadcast variable has been destroyed, it cannot be used again.

In [13]:
broad.destroy()
df.withColumn("State Full",udf_broad_func('State_Code')).show(5)

23/02/26 19:03:43 ERROR Utils: Exception encountered
org.apache.spark.SparkException: Attempted to use Broadcast(6) after it was destroyed (destroy at NativeMethodAccessorImpl.java:0) 
	at org.apache.spark.broadcast.Broadcast.assertValid(Broadcast.scala:144)
	at org.apache.spark.broadcast.TorrentBroadcast.$anonfun$writeObject$1(TorrentBroadcast.scala:223)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.Utils$.tryOrIOException(Utils.scala:1428)
	at org.apache.spark.broadcast.TorrentBroadcast.writeObject(TorrentBroadcast.scala:222)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at java.io.ObjectStreamClass.invokeWriteObject(ObjectStreamClass.java:1154)
	at java.io.ObjectOutputStream.writeSerialDat

Py4JJavaError: An error occurred while calling o63.showString.
: org.apache.spark.SparkException: Task not serializable
	at org.apache.spark.util.ClosureCleaner$.ensureSerializable(ClosureCleaner.scala:416)
	at org.apache.spark.util.ClosureCleaner$.clean(ClosureCleaner.scala:406)
	at org.apache.spark.util.ClosureCleaner$.clean(ClosureCleaner.scala:162)
	at org.apache.spark.SparkContext.clean(SparkContext.scala:2477)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$1(RDD.scala:860)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:414)
	at org.apache.spark.rdd.RDD.mapPartitions(RDD.scala:859)
	at org.apache.spark.sql.execution.python.EvalPythonExec.doExecute(EvalPythonExec.scala:90)
	at org.apache.spark.sql.execution.python.EvalPythonExec.doExecute$(EvalPythonExec.scala:87)
	at org.apache.spark.sql.execution.python.BatchE